# SECULOCK: SMART LOCKER SYSTEM

This project implements a smart locker security system using Raspberry Pi, Face Recognition, OTP Verification, and Intrusion Detection Mechanisms. The system ensures secure access control by verifying the user's identity through facial recognition using the DeepFace library and the FaceNet model. If face recognition fails, an OTP (One-Time Password) is sent to the owner's registered email for secondary authentication.

Additionally, the system incorporates vibration sensors to detect unauthorized access attempts. If a security breach is detected, an alert email with an intruder's photo is sent to the owner, and a buzzer is activated to deter the intruder.

## Key Features

✅ Face Recognition using DeepFace (FaceNet)

✅ OTP-based Authentication in case of face mismatch

✅ Intrusion Detection with vibration sensors

✅ Email Alert with intruder photo for unauthorized access

✅ LCD Display for real-time system status updates

This Jupyter Notebook contains the Python implementation of the SecuLock system, interfacing various hardware components like Raspberry Pi, USB Camera, 16x2 I2C LCD, 4x4 Matrix Keypad, Vibration Sensors, and Relays for real-time security monitoring.

## Detailed Explanation of Code

### Importing Libraries
This section imports the necessary libraries for face recognition, email handling, GPIO control, and LCD interaction.

In [ ]:
import json
import cv2
import numpy as np
import time
import smtplib
import random
import lgpio as GPIO
import smbus2 as smbus
from deepface import DeepFace
from email.mime.text import MIMEText
from email.mime.multipart import MIMEMultipart
from email.mime.base import MIMEBase
from email import encoders
import atexit

### Email Configuration
Defines the SMTP server details, sender credentials, recipient email, and email subject.

In [ ]:
smtp_port = 587  # SMTP port for email sending
smtp_server = "smtp.gmail.com"  # SMTP server address
email_from = "lockersender@gmail.com"  # Sender email address
email_to = "alanjoseph111aaa@gmail.com"  # Receiver email address
pswd = "jreesysnmnvqbfsr"  # Email password (should be securely stored)
SOURCE_IMAGES = ["Alan.jpeg", "Karthika.jpeg", "Nayana.jpeg", "Nebu.jpeg"]  # List of authorized faces
subject = "Theft detected !!"  # Subject of the alert email

### LCD Display Setup
Initializes the LCD display using I2C communication for displaying system status.

In [ ]:
# LCD Setup
I2C_ADDR = 0x27  # I2C address of the LCD module
bus = smbus.SMBus(1)  # Initialize the I2C bus

# LCD Control Constants
LCD_CHR = 1  # Mode for sending data
LCD_CMD = 0  # Mode for sending commands

# LCD Line Addresses
LINE_1 = 0x80  # Address for the first line of the LCD
LINE_2 = 0xC0  # Address for the second line of the LCD

# Enable and Backlight Control
ENABLE = 0b00000100  # Enable bit for LCD operation
BACKLIGHT = 0b00001000  # Backlight control bit

### LCD Display Functions
Defines functions for writing commands and text to the LCD display.

In [ ]:
def lcd_write(bits, mode):
    """Send command or data to the LCD"""
    high_bits = mode | (bits & 0xF0) | BACKLIGHT  # Extract the high nibble
    low_bits = mode | ((bits << 4) & 0xF0) | BACKLIGHT  # Extract the low nibble

    # Send high bits
    bus.write_byte(I2C_ADDR, high_bits)
    lcd_toggle_enable(high_bits)

    # Send low bits
    bus.write_byte(I2C_ADDR, low_bits)
    lcd_toggle_enable(low_bits)

def lcd_toggle_enable(bits):
    """Toggle the enable pin to latch data"""
    time.sleep(0.0005)  # Small delay
    bus.write_byte(I2C_ADDR, bits | ENABLE)  # Enable high
    time.sleep(0.0005)  # Small delay
    bus.write_byte(I2C_ADDR, bits & ~ENABLE)  # Enable low
    time.sleep(0.0005)  # Small delay

def lcd_init():
    """Initialize the LCD with the required commands"""
    lcd_write(0x33, LCD_CMD)  # Initialize LCD in 4-bit mode
    lcd_write(0x32, LCD_CMD)  # Set to 4-bit mode again (required)
    lcd_write(0x06, LCD_CMD)  # Set cursor move direction
    lcd_write(0x0C, LCD_CMD)  # Turn on display, hide cursor
    lcd_write(0x28, LCD_CMD)  # Set interface length (2-line, 5x7 dots)
    lcd_write(0x01, LCD_CMD)  # Clear display
    time.sleep(0.0005)  # Small delay after initialization

def lcd_display(text, line):
    """Display a given text on a specified LCD line"""
    lcd_write(line, LCD_CMD)  # Set cursor position to the specified line
    for char in text.ljust(16):  # Ensure text is exactly 16 characters
        lcd_write(ord(char), LCD_CHR)  # Write each character to the display

# Initialize the LCD at startup
lcd_init()

### Email Sending Functions
Functions to send alert emails with attachement and to send OTP.

In [ ]:
def send_emails(email_to):
    """Send an email alert with the captured face image when unauthorized access is detected."""
    try:
        body = "Unauthorized access attempt detected."  # Email body message
        msg = MIMEMultipart()  # Create a multipart email message
        msg['From'], msg['To'], msg['Subject'] = email_from, email_to, subject  # Set email headers
        msg.attach(MIMEText(body, 'plain'))  # Attach email body text

        # Attach the captured image
        filename = "captured_face.jpg"  # Name of the file to be attached
        with open(filename, 'rb') as attachment:  # Open image file in binary mode
            attachment_package = MIMEBase('application', 'octet-stream')  # Create a binary file package
            attachment_package.set_payload(attachment.read())  # Load image data into the package
            encoders.encode_base64(attachment_package)  # Encode the attachment
            attachment_package.add_header('Content-Disposition', f"attachment; filename= {filename}")  # Set file name
            msg.attach(attachment_package)  # Attach the image to the email

        # Establish connection with SMTP server and send email
        with smtplib.SMTP(smtp_server, smtp_port) as server:
            server.starttls()  # Secure the connection
            server.login(email_from, pswd)  # Log in to the email account
            server.sendmail(email_from, email_to, msg.as_string())  # Send email
            print(f"Email sent to: {email_to}")  # Print confirmation

    except Exception as e:
        print("Email sending error:", e)  # Print error if email sending fails


# Email for OTP
def otp_sent(subject, body):
    """Send an OTP email for verification."""
    try:
        msg = MIMEText(body)  # Create a plain text email
        msg['From'], msg['To'], msg['Subject'] = email_from, email_to, subject  # Set email headers

        # Establish connection with SMTP server and send email
        with smtplib.SMTP(smtp_server, smtp_port) as server:
            server.starttls()  # Secure the connection
            server.login(email_from, pswd)  # Log in to the email account
            server.send_message(msg)  # Send OTP email

    except Exception as e:
        print("Email error:", e)  # Print error if email sending fails
    

### OTP Generation
Generates and sends a one-time password (OTP) for verification.

In [ ]:
def send_otp():
    """Generate a 6-digit OTP and send it via email."""
    otp = str(random.randint(100000, 999999))  # Generate a random 6-digit OTP
    otp_sent("Your OTP for Verification", f"Your OTP is: {otp}")  # Send the OTP via email
    return otp  # Return the generated OTP for verification

### Face Detection and Recognition
Captures and compares a face using DeepFace's FaceNet model.

In [ ]:
# Face Capture and Comparison
def capture_face(frame):
    """Capture a face from the given video frame and save it as an image."""
    face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + "haarcascade_frontalface_default.xml")  
    # Load the pre-trained Haar cascade classifier for face detection

    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)  # Convert the frame to grayscale
    faces = face_cascade.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=5, minSize=(30, 30))  
    # Detect faces in the grayscale frame

    if len(faces) > 0:  # If at least one face is detected
        x, y, w, h = faces[0]  # Get coordinates of the first detected face
        face_roi = frame[y:y + h, x:x + w]  # Extract the face region from the original frame
        path = "captured_face.jpg"  # Define file path for the captured face
        cv2.imwrite(path, face_roi)  # Save the captured face as an image file
        return path  # Return the saved image path

    return None  # Return None if no face is detected

# Face Recognition Function
def compare_faces(source, captured):
    """Compare two face images using the DeepFace FaceNet model."""
    try:
        # Perform face verification using DeepFace
        result = DeepFace.verify(img1_path=source, img2_path=captured, model_name="FaceNet", enforce_detection=False)
        return result.get('verified', False)  # Return True if the faces match, otherwise False
    except Exception as e:
        print("Face comparison error:", e)  # Handle errors in face verification
        return False  # Return False if an error occurs

### GPIO Setup
Initializes GPIO pins for relays, buzzer, and vibration sensors.

In [ ]:
chip = GPIO.gpiochip_open(0)

def cleanup():
    """Release GPIO resources on exit"""
    print("Releasing GPIO resources...")
    GPIO.gpiochip_close(chip)

atexit.register(cleanup)

### GPIO Setup
Initializes the GPIO chip for controlling Raspberry Pi pins and ensures cleanup on exit.

In [ ]:
chip = GPIO.gpiochip_open(0)  # Open GPIO chip 0 for controlling Raspberry Pi GPIO pins

def cleanup():
    """Release GPIO resources upon program exit."""
    print("Releasing GPIO resources...")
    GPIO.gpiochip_close(chip)  # Close the GPIO chip to free resources

atexit.register(cleanup)  # Ensure cleanup() is called automatically on program exit

### Keypad Initialization
Defines the GPIO pins for a 4x4 keypad and configures them for input and output.

In [ ]:
ROWS = [17, 27, 22, 10]  # GPIO pins connected to the keypad rows
COLS = [9, 11, 5, 6]  # GPIO pins connected to the keypad columns

KEYPAD = [
    ['1', '2', '3', 'A'],
    ['4', '5', '6', 'B'],
    ['7', '8', '9', 'C'],
    ['*', '0', '#', 'D']
]

# Configure keypad row pins as OUTPUT (initially HIGH)
for row in ROWS:
    GPIO.gpio_claim_output(chip, row, 1)

# Configure keypad column pins as INPUT with pull-up resistors
for col in COLS:
    GPIO.gpio_claim_input(chip, col, GPIO.SET_PULL_UP)

### Keypad Read Function
Scans the keypad and returns the key pressed by the user.

In [ ]:
def read_keypad():
    """Read a key press from the keypad and return the corresponding character."""
    for row_index, row_pin in enumerate(ROWS):
        GPIO.gpio_write(chip, row_pin, 0)  # Set row LOW to detect key press
        for col_index, col_pin in enumerate(COLS):
            if GPIO.gpio_read(chip, col_pin) == 0:  # If column goes LOW (key press detected)
                time.sleep(0.2)  # Debounce delay
                GPIO.gpio_write(chip, row_pin, 1)  # Reset row back to HIGH
                return KEYPAD[row_index][col_index]  # Return pressed key
        GPIO.gpio_write(chip, row_pin, 1)  # Reset row before next scan
    return None  # Return None if no key is pressed

### GPIO Pin Initialization
Defines and configures the GPIO pins for the lock, buzzer, and vibration sensors.

In [ ]:
RELAY_LOCK_PIN, RELAY_BUZZER_PIN = 26, 19  # GPIO pins for electronic lock and buzzer
VIBRATION_SENSOR_1 = 23  # GPIO pin for Vibration Sensor 1
VIBRATION_SENSOR_2 = 24  # GPIO pin for Vibration Sensor 2

# Configure vibration sensor pins as INPUT with pull-up resistors
GPIO.gpio_claim_input(chip, VIBRATION_SENSOR_1, GPIO.SET_PULL_UP)
GPIO.gpio_claim_input(chip, VIBRATION_SENSOR_2, GPIO.SET_PULL_UP)

# Configure relay pins as OUTPUT and set them HIGH initially (inactive state)
GPIO.gpio_claim_output(chip, RELAY_LOCK_PIN, 1)
GPIO.gpio_claim_output(chip, RELAY_BUZZER_PIN, 1)

### Buzzer Function
Activates the buzzer for a specified duration (default: 5 seconds).

In [ ]:
def trigger_buzzer(duration=5):
    """Activate the buzzer for a specified duration (default: 5 seconds)."""
    GPIO.gpio_write(chip, RELAY_BUZZER_PIN, 0)  # Turn ON the buzzer (LOW active)
    time.sleep(duration)  # Wait for the specified duration
    GPIO.gpio_write(chip, RELAY_BUZZER_PIN, 1)  # Turn OFF the buzzer (HIGH inactive)

### Main Loop
Continuously monitors vibration sensors and handles access authentication via face recognition or OTP.

In [ ]:
while True:
    # Read the vibration sensor states
    sensor_1_state = GPIO.gpio_read(chip, VIBRATION_SENSOR_1)
    sensor_2_state = GPIO.gpio_read(chip, VIBRATION_SENSOR_2)

    # If any vibration sensor detects movement, trigger security alert
    if sensor_1_state == 0 or sensor_2_state == 0:
        trigger_buzzer()  # Activate the buzzer
        otp_sent("Theft Detected", "Vibration sensors activated")  # Send alert email
        time.sleep(0.1)  # Short delay before continuing
    
    # Display initial message on LCD prompting user to start
    lcd_display("Press A ", LINE_1)
    lcd_display("to start", LINE_2)
    
    # Read keypad input
    i = read_keypad()

### Face Matching
Compares captured face with stored images and grants access if matched.

In [ ]:
            # Compare captured face with stored faces
            if captured_path and any(compare_faces(img, captured_path) for img in SOURCE_IMAGES):
                lcd_display("Access Granted", LINE_1)
                GPIO.gpio_write(chip, RELAY_LOCK_PIN, 0)  # Unlock the locker
                time.sleep(1)

                # Prompt user to close locker
                while True:
                    lcd_display("Press B", LINE_1)
                    lcd_display("to lock", LINE_2)
                    close = read_keypad()
                    if close == "B":  # Close the locker when 'B' is pressed
                        lcd_display("Closed", LINE_1)
                        GPIO.gpio_write(chip, RELAY_LOCK_PIN, 1)  # Lock the locker
                        break  # Exit locker close loop
                break  # Exit face recognition loop

### OTP Verification
If face recognition fails, prompts the user to enter an OTP for access.

In [ ]:
            else:  # If face is not recognized, proceed with OTP verification
                lcd_display("Face not matched", LINE_1)
                sent_otp = send_otp()  # Generate and send OTP
                lcd_display("OTP Sent", LINE_1)
                print("OTP sent")
                time.sleep(1)
                lcd_display("Enter OTP:", LINE_1)

                input_sequence = ""  # Variable to store entered OTP
                otp_verified = False  # Flag to track OTP verification status

### OTP Input Handling
Reads user input and grants or denies access based on OTP correctness.

In [ ]:
                # OTP input loop
                while True:
                    key = read_keypad()  # Read key press
                    if key:
                        if key == "#":  # If '#' is pressed, verify OTP
                            if input_sequence == sent_otp:
                                lcd_display("Access Granted", LINE_1)
                                time.sleep(1)
                                lcd_display("", LINE_1)
                                lcd_display("", LINE_2)
                                GPIO.gpio_write(chip, RELAY_LOCK_PIN, 0)  # Unlock locker
                                otp_verified = True  # Mark OTP as verified
                            else:
                                lcd_display("Access Denied", LINE_1)
                                trigger_buzzer()  # Alert unauthorized access
                                send_emails(email_to)  # Send intruder alert email
                            break  # Exit OTP input loop
                        
                        elif key == "*":  # If '*' is pressed, clear OTP input
                            input_sequence = ""
                        else:  # Otherwise, append digit to OTP input
                            input_sequence += key
                            lcd_display(input_sequence, LINE_2)  # Display entered OTP
                            print(key)

### Locker Closing
Ensures the locker is securely closed after successful authentication.

In [ ]:
                if otp_verified:  # If OTP was correct, allow locker closure
                    while True:
                        lcd_display("Press B", LINE_1)
                        lcd_display("to lock", LINE_2)
                        close2 = read_keypad()
                        if close2 == "B":  # Close locker when 'B' is pressed
                            lcd_display("Closed", LINE_1)
                            GPIO.gpio_write(chip, RELAY_LOCK_PIN, 1)  # Lock locker
                            break  # Exit locker close loop
                break  # Exit face recognition loop

        cap.release()  # Release webcam
        cv2.destroyAllWindows()  # Close OpenCV windows